In [2]:
import inspect

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import spark as spark

In [3]:
df = pd.read_excel("posts.xlsx")
df.head()

,Unnamed: 0,id,date,text,publisher,reaction,comment
0,0,95367da2-8313-49c4-81f5-cbb1f8af69be,2023-01-14 17:17:08.793,"Что происходит на Мосбирже, останется на Мосби...",Pulse_Official,161,111.0
1,1,c179933b-b992-41a8-953c-e8cddce0a277,2023-01-13 16:30:00.000,В прямом эфире Василий Олейник и Ирина Ахмадул...,Tinkoff_Investments,140,29.0
2,2,9ee444e3-0ff1-4622-82b9-6ff7b1d00ec7,2023-01-12 06:50:00.000,Доброе утро! \nПродолжаем говорить на тематику...,Invest_Dim,315,149.0
3,3,4963f23d-f3ef-4c53-903e-74f62b851aab,2023-01-13 16:25:00.000,"Российский рынок \n\nНапоминаем, что по акциям...",Tinkoff_Investments,67,15.0
4,4,bedb0a0f-7657-4d27-a1fd-0b8005813eb5,2023-01-12 13:41:00.000,📊 Инфляционные ожидания: что такое и зачем их ...,FinDay,105,12.0


In [4]:
df.describe()

,Unnamed: 0,reaction,comment
count,3450.000000,3450.000000,3418.000000
mean,1724.500000,161.273043,177.120538
std,996.073541,147.782744,937.298884
min,0.000000,2.000000,1.000000
25%,862.250000,65.000000,16.000000
50%,1724.500000,129.000000,40.000000
75%,2586.750000,214.000000,91.000000
max,3449.000000,1672.000000,12891.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3450 entries, 0 to 3449
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  3450 non-null   int64         
 1   id          3450 non-null   object        
 2   date        3450 non-null   datetime64[ns]
 3   text        3450 non-null   object        
 4   publisher   3450 non-null   object        
 5   reaction    3450 non-null   int64         
 6   comment     3418 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 148.3+ KB


In [6]:
df['text'] = df['text'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3450 entries, 0 to 3449
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  3450 non-null   int64         
 1   id          3450 non-null   object        
 2   date        3450 non-null   datetime64[ns]
 3   text        3450 non-null   object        
 4   publisher   3450 non-null   object        
 5   reaction    3450 non-null   int64         
 6   comment     3418 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 148.3+ KB


In [7]:
import re

def standardize_text(text):
    text = re.sub('([$A-Z])', '', text)
    text = text.lower()
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('http[^\s]+', '', text)
    text = re.sub('\$\d+\.\d{2}', '', text)
    #text = re.sub("[^0-9(),!?@\'\`\"\_\n]", " ", text) ломает текст
    text = re.sub(r'\d+', '', text)  # убирает цифры
    text = re.sub("[^\w\s]"," ", text)
    text = re.sub('\n', ' ', text)
    return text



In [8]:
df['text'][21]


'В США замедляется инфляция \n\nПоказатель инфляции в США в годовом выражении за декабрь составил 6,5%. Данные совпали с ожиданиями аналитиков. Это самый низкий уровень с октября 2021 года.\n\nБазовая инфляция (она рассчитывается без учета цен на продукты питания и энергоносители) снизилась до 5,7%. Прогноз также совпал.\n\nНапомним, инфляция за ноябрь была на уровне 7,1% в годовом выражении, в октябре — 7,7%.\n\nКомментарий аналитика Тинькофф Инвестиций Михаила Иванова\n\nКак мы и писали в своей стратегии на этот год, инфляция продолжает замедляться. Этому во многом способствует резкий рост ключевой ставки, который тормозит экономику. Мы по-прежнему ожидаем, что инфляция в США по итогам 2023 года опустится до уровня около 4-5%. В целом сегодняшние значения были ожидаемы рынком, поэтому не принесли существенных изменений в ожидания по ключевой ставке. Рынок ждет, что на ближайшем заседании регулятор замедлит шаг повышения ставки до 25 б. п.\n\n#новости #news\n\nСтратегия: https://www.t

In [9]:
standardize_text(df['text'][21])

'в сша замедляется инфляция   показатель инфляции в сша в годовом выражении за декабрь составил     данные совпали с ожиданиями аналитиков  это самый низкий уровень с октября  года   базовая инфляция  она рассчитывается без учета цен на продукты питания и энергоносители  снизилась до     прогноз также совпал   напомним  инфляция за ноябрь была на уровне    в годовом выражении  в октябре        комментарий аналитика тинькофф инвестиций михаила иванова  как мы и писали в своей стратегии на этот год  инфляция продолжает замедляться  этому во многом способствует резкий рост ключевой ставки  который тормозит экономику  мы по прежнему ожидаем  что инфляция в сша по итогам  года опустится до уровня около     в целом сегодняшние значения были ожидаемы рынком  поэтому не принесли существенных изменений в ожидания по ключевой ставке  рынок ждет  что на ближайшем заседании регулятор замедлит шаг повышения ставки до  б  п    новости  news  стратегия  '